# Pytorch를 이용한 간단한 CNN 구현

## 1. 필요한 라이브러리 불러오기

In [1]:
import torch    # Pytorch 기본 라이브러리
import torch.nn as nn   # Neural Network (nn)에 관련된 모듈들
from torch.utils.data import DataLoader # Database의 데이터들을 불러오기 위한 라이브러리

from torchvision import transforms, datasets    # 불러온 입력을 처리하는 모듈들

import os   # 경로 탐색, 접근 등을 처리하기 위한 라이브러리
import numpy as np # 행렬 연산에 사용하는 라이브러리

## 2. 학습을 위한 하이퍼 파리미터 설정

In [2]:
lr = 0.01
batch_size = 128
num_epoch = 5

ckpt_dir = './checkpoint'   # 학습된 파라미터를 저장할 경로

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
print(device)

cpu


## 3. CNN 구축

In [5]:
class Net(nn.Module):       # nn.Module를 상속받아서 사용.
    def __init__(self):
        super(Net, self).__init__() # 상속해준 부모 클래스의 Method를 사용할 때 super(하위클래스, 하위클래스의 객체)

        # Block 1
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=10, 
                               kernel_size=5, 
                               stride=1, 
                               padding=0)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)    # cf. AvgPool2d

        # Block 2
        self.conv2 = nn.Conv2d(in_channels=10, 
                               out_channels=20, 
                               kernel_size=5, 
                               stride=1, 
                               padding=0)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)    # H * W * C = 320

        self.fc1 = nn.Linear(in_features=320, 
                             out_features=50)             # Fully-Connected Layer
        self.relu1_fc1 = nn.ReLU()

        self.fc2 = nn.Linear(in_features=50, out_features=10, bias=True)

    def forward(self, x):   # x: 입력영상 (torch.Tensor())
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = x.view(-1, 320)     # x: B=128, C=20, H=4, W=4

        x = self.fc1(x)
        x = self.relu1_fc1(x)

        x = self.fc2(x)

        return x

## 4. 학습된 파라미터를 저장하거나 불러오는 함수 작성

In [6]:
def save(ckpt_dir, net, optim, epoch):      # ckpt_dir: checkpoint를 저장할 경로, net, optim, epoch
    if not os.path.exists(ckpt_dir):    # ckpt_dir이 존재하는지 확인하는 함수
        os.makedirs(ckpt_dir)           # 디렉토리를 만들어주는 함수

    torch.save({'net': net.state_dict(),        # 네트워크에 있는 변수들
                'optim': optim.state_dict()},   # optimizer에 있는 변수들
               './%s/model_epoch%d.pth' % (ckpt_dir, epoch))

def load(ckpt_dir, net, optim):
    ckpt_lst = os.listdir(ckpt_dir)  # 입력한 디렉토리 내의 모든 파일과 디렉토리의 리스트를 반환
    ckpt_lst.sort()  # 정렬

    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])

    return net, optim

서식문자를 이용한 문자열 표현

In [7]:
print('%s is %s.' % ('Cat', 'animal'))
print('%d + %d is %d' % (2, 3, 5))

Cat is animal.
2 + 3 is 5


In [8]:
a = ['b', 'a', 'c']
a.sort()
print(a)

['a', 'b', 'c']


## 5. Database 불러오기
### Database가 없을 경우 Pytorch에 내장되어 있는 MNIST 다운로드 함수 사용

In [9]:
transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize(mean=(0.5,), std=(0.5,))])

dataset = datasets.MNIST(download=True, 
                         root='./', 
                         train=True, 
                         transform=transform)   # 개별 데이터 파일의 처리를 담당
loader = DataLoader(dataset, 
                    batch_size=batch_size, 
                    shuffle=True, 
                    num_workers=0)  # 미니배치 처리를 담당

num_data = len(loader.dataset)
num_batch = num_data // batch_size

In [10]:
print(num_data)

60000


## 6. 네트워크 및 손실함수 선언

In [11]:
net = Net()
net = net.to(device)
params = net.parameters()

fn_loss = nn.CrossEntropyLoss().to(device)

def fn_pred(output):
    return torch.softmax(output, dim=1)     # B=128 (dim=0), 10 (dim=1)

def fn_accuracy(pred, label):
    return (pred.max(dim=1)[1] == label).type(torch.float).mean()


## 7. Optimizer 선언

In [12]:
optimizer = torch.optim.SGD(params, lr=lr)

## 8. 학습 진행

In [13]:
for epoch in range(1, num_epoch + 1):
    net.train() # net.eval()

    loss_arr = []
    acc_arr = []

    for batch, (input, label) in enumerate(loader):
        input = input.to(device)
        label = label.to(device)

        output = net(input)     # net.forward(input)
        pred = fn_pred(output)  # softmax

        optimizer.zero_grad()   # G = 0 

        loss = fn_loss(output, label)   # 
        acc = fn_accuracy(pred, label)

        loss.backward() # 그라디언트를 계산

        optimizer.step()    # 그라디언트를 사용하여 파라미터를 업데이트

        loss_arr += [loss.item()]
        acc_arr += [acc.item()]

        print('TRAIN: EPOCH %04d/%04d | BATCH %04d/%04d | LOSS: %.4f | ACC %.4f' %
              (epoch, num_epoch, batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))

    save(ckpt_dir = ckpt_dir, net=net, optim=optimizer, epoch=epoch)

TRAIN: EPOCH 0001/0005 | BATCH 0000/0468 | LOSS: 2.2929 | ACC 0.1328
TRAIN: EPOCH 0001/0005 | BATCH 0001/0468 | LOSS: 2.2890 | ACC 0.1406
TRAIN: EPOCH 0001/0005 | BATCH 0002/0468 | LOSS: 2.2940 | ACC 0.1276
TRAIN: EPOCH 0001/0005 | BATCH 0003/0468 | LOSS: 2.2978 | ACC 0.1113
TRAIN: EPOCH 0001/0005 | BATCH 0004/0468 | LOSS: 2.2983 | ACC 0.1172
TRAIN: EPOCH 0001/0005 | BATCH 0005/0468 | LOSS: 2.2978 | ACC 0.1198
TRAIN: EPOCH 0001/0005 | BATCH 0006/0468 | LOSS: 2.2970 | ACC 0.1217
TRAIN: EPOCH 0001/0005 | BATCH 0007/0468 | LOSS: 2.2967 | ACC 0.1191
TRAIN: EPOCH 0001/0005 | BATCH 0008/0468 | LOSS: 2.2957 | ACC 0.1259
TRAIN: EPOCH 0001/0005 | BATCH 0009/0468 | LOSS: 2.2965 | ACC 0.1281
TRAIN: EPOCH 0001/0005 | BATCH 0010/0468 | LOSS: 2.2971 | ACC 0.1314
TRAIN: EPOCH 0001/0005 | BATCH 0011/0468 | LOSS: 2.2964 | ACC 0.1328
TRAIN: EPOCH 0001/0005 | BATCH 0012/0468 | LOSS: 2.2953 | ACC 0.1346
TRAIN: EPOCH 0001/0005 | BATCH 0013/0468 | LOSS: 2.2952 | ACC 0.1350
TRAIN: EPOCH 0001/0005 | BATCH 001

KeyboardInterrupt: 